## Climbing Shoe Website Parser
## Website: TrackNTrail

In [51]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pyshorteners
import pyperclip
import json

Parse for max number of pages to iterate the parser through

In [83]:
url = "https://shop.blocshop.com/collections/all-climbing-shoes?page=3&section_id=template--15175211253894__main-collection"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find("div", class_="collection page-width page-width--full relative with-drawer")
product_elements = results.find_all("div", class_="card product-card product-card--card grid leading-none relative overflow-hidden")

for sale_element in product_elements:
    if sale_element.find("span", class_="price__sale inline-flex items-center h-full relative") != None:
        if sale_element.find("span", class_="price__sale inline-flex items-center h-full relative") != None:
            if len(sale_element.find("span", class_="price__regular whitespace-nowrap").text.strip()) > 4:
                a = float(sale_element.find("span", class_="price__regular whitespace-nowrap").text.strip()[6:]),
                print(a)

# print(results)

(151.0,)


Parsing through each webpage and converting it into a dataframe

In [80]:
df = pd.DataFrame(columns=["Shoe","Sale Price","Original Price","Discount","URL","Shipping Cost"])

for i in range(1,9):
    url = "https://shop.blocshop.com/collections/all-climbing-shoes?page={}&section_id=template--15175211253894__main-collection".format(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("div", class_="collection page-width page-width--full relative with-drawer")
    product_elements = results.find_all("div", class_="card product-card product-card--card grid leading-none relative overflow-hidden")

    for sale_element in product_elements:
        if sale_element.find("span", class_="price__sale inline-flex items-center h-full relative") != None:
            if len(sale_element.find("span", class_="price__regular whitespace-nowrap").text.strip()) > 4:
                new_row = {"Shoe" : sale_element.find("div", class_="product-card__top w-full").find("a").text.strip() 
                        + " " 
                        + sale_element.find("div", class_="product-card__details flex flex-col lg:flex-row items-baseline gap-2 w-full").find("a").text.strip(),
                "Sale Price" : float(sale_element.find("span", class_="price__regular whitespace-nowrap").text.strip()[6:]),
                "Original Price" : float(sale_element.find("span", class_="price__sale inline-flex items-center h-full relative").text.strip()[1:]),
                "Discount" : str(round((float(sale_element.find("span", class_="price__sale inline-flex items-center h-full relative").text.strip()[1:])
                                        -float(sale_element.find("span", class_="price__regular whitespace-nowrap").text.strip()[6:]))
                                    /float(sale_element.find("span", class_="price__sale inline-flex items-center h-full relative").text.strip()[1:]),2)*100)+"%",
                "URL" : "shop.blocshop.com" + sale_element.find("a")["href"],
                "Shipping Cost" : 0
                }
                new_row_df = pd.DataFrame([new_row])
                df = pd.concat([df, new_row_df],ignore_index=True)
            else:
                new_row = {"Shoe" : sale_element.find("div", class_="product-card__top w-full").find("a").text.strip() 
                        + " " 
                        + sale_element.find("div", class_="product-card__details flex flex-col lg:flex-row items-baseline gap-2 w-full").find("a").text.strip(),
                "Sale Price" : float(sale_element.find("span", class_="price__regular whitespace-nowrap").text.strip()[1:]),
                "Original Price" : float(sale_element.find("span", class_="price__sale inline-flex items-center h-full relative").text.strip()[1:]),
                "Discount" : str(round((float(sale_element.find("span", class_="price__sale inline-flex items-center h-full relative").text.strip()[1:])
                                        -float(sale_element.find("span", class_="price__regular whitespace-nowrap").text.strip()[1:]))
                                    /float(sale_element.find("span", class_="price__sale inline-flex items-center h-full relative").text.strip()[1:]),2)*100)+"%",
                "URL" : "shop.blocshop.com" + sale_element.find("a")["href"],
                "Shipping Cost" : 0
                }
                new_row_df = pd.DataFrame([new_row])
                df = pd.concat([df, new_row_df],ignore_index=True)


type_tiny = pyshorteners.Shortener() 
df["URL"] = df["URL"].apply(lambda x: type_tiny.tinyurl.short(x))
df["Post-Tax Price"] = (df["Sale Price"]+df["Shipping Cost"])*1.13

ValueError: could not convert string to float: ''

Output Table

In [9]:
#Output the table with values sorted
df.sort_values(by="Discount",ascending=False)

,Shoe,Sale Price,Original Price,Discount,URL,Shipping Cost,Post-Tax Price
0,Zone LV Shoe,129.99,174.95,26.0%,https://tinyurl.com/27y8q3yq,25,175.1387
1,The General,169.99,229.95,26.0%,https://tinyurl.com/2xtudhgv,25,220.3387
